## Assignment 1 Task 2

<strong>Name:</strong> Elroy Chua Ming Xuan </br>
<strong>UOW ID: </strong> 7431673 </br>
<strong>Data set: </strong> https://www.kaggle.com/datasets/muhammadshahidazeem/customer-churn-dataset

#### Implement three DT models by the split criteria of Information Gain, Gain Ratio and Gini Index. 
You can use either binary-split or multiple-split.
Decision Tree models based on Information Gain, Gain Ratio, and Gini Index, you need to implement three different splitting criteria. All these criteria basically help in selecting the best feature for splitting the data.

In [69]:
import numpy as np
import pandas as pd
import math
from pprint import pprint


In [70]:
df_train = pd.read_csv("customer_churn_dataset-training-master.csv")
df_test = pd.read_csv("customer_churn_dataset-testing-master.csv")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440833 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  object 
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  object 
 8   Contract Length    440832 non-null  object 
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), object(3)
memory usage: 40.4+ MB


In [71]:
df_train.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [72]:
df_test.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0


#### Pre-process the data
1. Drop null values
2. Remove CustomerID column
3. Add new column support request frequency

In [73]:
# Drop the CustomerID column and remove all rows with missing values
df_train = df_train.dropna()
df_train = df_train.drop('CustomerID', axis=1)
df_test = df_test.drop('CustomerID', axis=1)
# Adding a new column to the dataframe
df_train["Support request frequency"] = df_train["Support Calls"] / df_train["Tenure"]
df_test["Support request frequency"] = df_test["Support Calls"] /  df_test["Tenure"]
# Using 500000 rows for training and testing
df_train = df_train.head(250000)
df_test = df_test.head(250000)


In [74]:
# Check for null values in training data
df_train.isnull().sum()


Age                          0
Gender                       0
Tenure                       0
Usage Frequency              0
Support Calls                0
Payment Delay                0
Subscription Type            0
Contract Length              0
Total Spend                  0
Last Interaction             0
Churn                        0
Support request frequency    0
dtype: int64

In [75]:
# Check for null values in test data
df_test.isnull().sum()


Age                          0
Gender                       0
Tenure                       0
Usage Frequency              0
Support Calls                0
Payment Delay                0
Subscription Type            0
Contract Length              0
Total Spend                  0
Last Interaction             0
Churn                        0
Support request frequency    0
dtype: int64

In [76]:
df_train.describe()


,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn,Support request frequency
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.624540,30.464540,15.470430,5.097880,15.14379,544.615180,15.549900,0.986720,0.399172
std,13.934461,17.345644,8.684231,3.151457,8.96837,259.687842,8.648444,0.114472,0.910475
min,18.000000,1.000000,1.000000,0.000000,0.00000,100.000000,1.000000,0.000000,0.000000
25%,29.000000,15.000000,8.000000,2.000000,7.00000,320.000000,8.000000,1.000000,0.076923
50%,42.000000,30.000000,15.000000,5.000000,15.00000,541.000000,16.000000,1.000000,0.166667
75%,54.000000,45.000000,23.000000,8.000000,23.00000,769.000000,23.000000,1.000000,0.333333
max,65.000000,60.000000,30.000000,10.000000,30.00000,1000.000000,30.000000,1.000000,10.000000


In [77]:
df_test.describe()

,Age,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn,Support request frequency
count,64374.000000,64374.000000,64374.000000,64374.000000,64374.000000,64374.000000,64374.000000,64374.000000,64374.000000
mean,41.970982,31.994827,15.080234,5.400690,17.133952,541.023379,15.498850,0.473685,0.372214
std,13.924911,17.098234,8.816470,3.114005,8.852211,260.874809,8.638436,0.499311,0.853217
min,18.000000,1.000000,1.000000,0.000000,0.000000,100.000000,1.000000,0.000000,0.000000
25%,30.000000,18.000000,7.000000,3.000000,10.000000,313.000000,8.000000,0.000000,0.089286
50%,42.000000,33.000000,15.000000,6.000000,19.000000,534.000000,15.000000,0.000000,0.170213
75%,54.000000,47.000000,23.000000,8.000000,25.000000,768.000000,23.000000,1.000000,0.300000
max,65.000000,60.000000,30.000000,10.000000,30.000000,1000.000000,30.000000,1.000000,10.000000


#### Correlation Matrix to find feature columns

In [78]:
sub_data = df_train.select_dtypes(include=['int', 'float']).columns
corr_mattrix = df_train[sub_data].corr()
# Compare with label column
abs(corr_mattrix['Churn']).sort_values(ascending=False)


Churn                        1.000000
Support Calls                0.130339
Total Spend                  0.092082
Payment Delay                0.067396
Age                          0.048514
Support request frequency    0.037806
Last Interaction             0.037185
Tenure                       0.012601
Usage Frequency              0.010555
Name: Churn, dtype: float64

#### Check purity function

In [79]:
def check_purity(data):
    class_col = data[:, -1]
    unique_class = np.unique(class_col)
    if len(unique_class) == 1:
        return True
    return False


In [80]:
#### classify function
def classify(data):
    class_col = data[:,-1]
    unique_class, count_class = np.unique(class_col, return_counts=True)
    major_class = np.argmax(count_class)
    return unique_class[major_class]

### Train Test Split Function

In [81]:
def split_check(data):
    potential = {}
    _, feature_num = np.shape(data)
    for feature in range(feature_num - 1):
        potential[feature] = []
        value_list = np.unique(data[:, feature])

        # For categorical values
        if isinstance(value_list[0], str):
            for val in value_list:
                potential[feature].append(val)
        else:
            # For continuous values
            for index in range(len(value_list)):
                if index != 0:
                    current_val = value_list[index]
                    previous_val = value_list[index - 1]
                    potential[feature].append((current_val + previous_val) / 2)
    return potential


In [82]:
def split(data, feature, value):
    feature_list = data[:, feature]
    if isinstance(feature_list[0], str):
        data_left = data[feature_list == value]
        data_right = data[feature_list != value]
    else:
        data_left = data[feature_list <= value]
        data_right = data[feature_list > value]
    return data_left, data_right


#### Calculate Entropy

In [83]:
def Entropy(data):
    label_col = data[:, -1]
    _, value_count = np.unique(label_col, return_counts=True)
    ent = 0.0
    for i in value_count:
        probabilities = i / sum(value_count)
        ent += - probabilities * math.log(probabilities, 2)
    return ent


#### Calculate total entropy (IntrinsicInfo) of the dataset

In [84]:
def total_entropy(data_left, data_right):
    total_length = len(data_left) + len(data_right)
    probabilities_left = len(data_left) / total_length
    probabilities_right = len(data_right) / total_length
    total_ent = probabilities_left * \
        Entropy(data_left) + probabilities_right * Entropy(data_right)
    return total_ent


#### Calculate Information Gain

In [85]:
def InfoGainSplit(data, potential_split):
    IGMAX = -1.0
    split_key = -1
    split_val = -1
    for key, value_list in potential_split.items():
        for value in value_list:
            data_left, data_right = split(data, key, value)
            IG = Entropy(data) - total_entropy(data_left, data_right)
            if IG >= IGMAX:
                IGMAX = IG
                split_key = key
                split_val = value
    return split_key, split_val


#### Calculate Gini Index

In [86]:
def Gini(data):
    label_col = data[:, -1]
    _, value_count = np.unique(label_col, return_counts=True)
    gini = 0.0
    for i in value_count:
        probabilities = i / sum(value_count)
        gini += probabilities * (1 - probabilities ** 2)
    return gini


In [87]:
def TotalGini(data_left, data_right):
    total_length = len(data_left) + len(data_right)
    probabilities_left = len(data_left) / total_length
    probabilities_right = len(data_right) / total_length
    total_gini = probabilities_left * \
        Gini(data_left) + probabilities_right * Gini(data_right)
    return total_gini


#### Decision Tree Implementation

In [88]:
def DT(data, model):
    # base case
    if check_purity(data):
        return classify(data)
    else:
        potential_split = split_check(data)
        if model == "Info Gain":
            split_key, split_value = InfoGainSplit(data, potential_split)

        data_left, data_right = split(data, split_key, split_value)
        question = "{} <= {}".format(split_key, split_value)
        subtree = {question: []}

        yes_ans = DT(data_left, model)
        no_ans = DT(data_right, model)

        subtree[question].append(yes_ans)
        subtree[question].append(no_ans)
    return subtree


In [89]:
cleaned_data = df_train[['Age', 'Support Calls', 'Payment Delay',
                           'Total Spend', 'Support request frequency', 'Last Interaction', 'Churn']]
cleaned_data_test = df_test[['Age', 'Support Calls', 'Payment Delay',
                               'Total Spend', 'Support request frequency', 'Last Interaction', 'Churn']]


#### Info Gain Tree

In [90]:
tree = DT(cleaned_data.values, 'Info Gain')
tree


{'1 <= 4.5': [{'3 <= 499.5': [1.0,
    {'2 <= 20.5': [{'0 <= 50.5': [{'5 <= 15.5': [{'1 <= 2.5': [{'4 <= 0.04040816326530612': [{'5 <= 1.5': [{'3 <= 591.0': [1.0,
                  {'0 <= 41.5': [{'4 <= 0.03478523895946763': [{'0 <= 21.5': [{'3 <= 650.5': [1.0,
                          0.0]},
                        {'2 <= 8.5': [{'0 <= 30.5': [{'3 <= 749.5': [0.0,
                              {'3 <= 981.0': [1.0, 0.0]}]},
                            0.0]},
                          {'3 <= 597.5': [0.0,
                            {'0 <= 31.5': [{'0 <= 29.5': [{'2 <= 19.5': [1.0,
                                  0.0]},
                                {'4 <= 0.00847457627118644': [0.0, 1.0]}]},
                              1.0]}]}]}]},
                      1.0]},
                    {'3 <= 682.5': [{'3 <= 664.5': [1.0, 0.0]}, 1.0]}]}]},
                {'2 <= 19.5': [{'5 <= 5.5': [{'3 <= 569.5': [{'3 <= 542.5': [{'1 <= 1.5': [{'4 <= 0.00847457627118644': [{'0 <= 35.5': [1.0,
      

In [91]:
def classify_entry(data_entry, decision_tree):
    # Extract the question from the current node of the tree
    current_question = list(decision_tree.keys())[0]
    feature_name, comparison_operator, threshold_value = current_question.split()

    # Convert the threshold_value back to its original type (either int/float or string)
    try:
        threshold_value = float(threshold_value)
    except ValueError:
        pass

    # Determine which subtree to follow based on the question
    if isinstance(threshold_value, str):  # Categorical
        if data_entry[feature_name] == threshold_value:
            subtree = decision_tree[current_question][0]
        else:
            subtree = decision_tree[current_question][1]
    else:  # Continuous
        if data_entry[feature_name] <= threshold_value:
            subtree = decision_tree[current_question][0]
        else:
            subtree = decision_tree[current_question][1]

    # Base case: if we've reached a leaf node (i.e., subtree is not a dictionary)
    if not isinstance(subtree, dict):
        return subtree
    else:
        # If not a leaf node, continue classifying with the subtree
        return classify_entry(data_entry, subtree)


#### Output cleaned data

In [92]:
cleaned_data

,Age,Support Calls,Payment Delay,Total Spend,Support request frequency,Last Interaction,Churn
0,30.0,5.0,18.0,932.0,0.128205,17.0,1.0
1,65.0,10.0,8.0,557.0,0.204082,6.0,1.0
2,55.0,6.0,18.0,185.0,0.428571,3.0,1.0
3,58.0,7.0,7.0,396.0,0.184211,29.0,1.0
4,23.0,5.0,8.0,617.0,0.156250,20.0,1.0
...,...,...,...,...,...,...,...
99995,20.0,0.0,26.0,305.0,0.000000,5.0,1.0
99996,46.0,6.0,0.0,547.0,0.105263,27.0,1.0
99997,21.0,5.0,11.0,232.0,0.135135,23.0,1.0
99998,62.0,10.0,26.0,332.0,0.555556,14.0,1.0
